# Shopee Code League: Order Brushing

In [1]:
import pandas as pd
from tqdm import tqdm_notebook
from collections import defaultdict 
from datetime import datetime, timedelta

In [2]:
data = pd.read_csv('./order_brush_order.csv')

# Data Observation

In [3]:
data.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [4]:
data.event_time.loc[0]

'2019-12-27 00:23:03'

# Code

In [5]:
# create dictionary default to list if non-initiated key is called
shops_dict = defaultdict(list)

for i in tqdm_notebook(range(len(data))):
    oid, sid, uid, event_time = data.loc[i]
    time = datetime.strptime(event_time, '%Y-%m-%d %H:%M:%S')
    shops_dict[sid].append((time,uid,oid))

In [6]:
mot_gio = timedelta(hours=1.)
hai_gio = timedelta(hours=2.)

def detect(orders,pivot,pointer):
    assert(pivot <= pointer and pointer < len(orders) and pivot > 0)
    seg = defaultdict(list)
    
    for i in range(pivot, pointer+1):
        uid = orders[i][1]
        oid = orders[i][2]
        seg[uid].append(oid)
    
    brushing = (pointer-pivot+1 >= 3*len(seg)) # number of orders >= ( 3 * number of unique users )
    return brushing, seg

'''
param: orders
    orders: a list of tuples of record of orders of the shop
        in a shape of (time, userid, orderid) -> index = (0,1,2)
'''
def scan_through(orders):
    brushing_order_dict = {} # make sure no duplicated orders
    brushing_occurs = False # flag
    
    # get the first and the last of the sorted orders
    orders_head = orders[0][0]
    orders_tail = orders[-1][0]
    
    # insert two placeholder into the head and tail of the orders
    # which have time far enough from the head and the tail
    header = (orders_head + hai_gio,)
    footer = (orders_tail + hai_gio,)
    orders.insert(0,header)
    orders.append(  footer)
    
    for pivot in range(1, len(orders)-1): # don't include the header and footer
        for pointer in range(pivot, len(orders)-1):
            if orders[pointer][0] - orders[pivot][0] <= mot_gio :
                if (orders[pointer+1][0] - orders[pivot-1][0] > mot_gio and
                    orders[pointer+1][0] != orders[pointer][0] and
                    orders[pivot][0] != orders[pivot-1][0]):
                    brushing, seg = detect(orders,pivot,pointer)
                    
                    if brushing:
                        brushing_occurs = True
                        for uid, oid_list in seg.items():
                            for order in oid_list:
                                brushing_order_dict[order] = uid
                                
    suspicious_users = defaultdict(int)
    
    maxim = -1
    for oid, uid in brushing_order_dict.items():
        suspicious_users[uid] += 1
        maxim = max(maxim, suspicious_users[uid])
        
    candidates = []
    
    for uid, count in suspicious_users.items():
        if (count == maxim):
            candidates.append(uid)
    
    if len(candidates)==0:
        return brushing_occurs, '0'
    else:
        candidates.sort()
        return brushing_occurs, "&".join([str(uid) for uid in candidates])

In [7]:
result = []
for sid, orders in shops_dict.items():
    orders.sort()
    brushing_occurs, report_string = scan_through(orders)
    result.append((sid, report_string))
    
df = pd.DataFrame(result, columns= ['shopid', 'userid'])
df.to_csv('result.csv', index=False, header=True)